# Работа с таблицами в pandas

## Вступление
Сегодня мы начнём работать с табличными данными в питоне, и в этом нам поможет библиотека pandas. Pandas — это самый популярный инструмент для работы с данными. Цель семинара — познакомить вас с основными конструкциями pandas и научить им пользоваться.

Почему pandas удобный?
1.	Удобное представление таблиц и куча готовых методов, как работать с этими таблицами
2.	Простота использования
3.	Наличие встроенных визуализаций
4.	Наличие всех функций из excel (и даже больше)
5.	Универсальность инструмента, можно читать почти все табличные типы данных
6.	Хорошо подходит для экспериментов с данными

А почему он неудобный?
1.	Медленный
2.	Чаще всего нельзя использовать в проде из-за скорости
3.	Иногда сложно интерпретировать код, написанный на pandas

### План семинара
1. Работаем с датасетом оценок студентов
2. Работаем с датасетом пассажиров Титаника

### Дополнительные материалы

[Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)

[10 Minutes To Pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html)

[Pandas CookBook](https://pandas.pydata.org/pandas-docs/stable/cookbook.html#cookbook)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## 1. Работаем с датасетом оценок студентов

По ссылкам можно скачать [датасет с оценками](https://disk.yandex.ru/d/86NPUWOVSS13og) и [таблицу с именами и фамилиями](https://disk.yandex.ru/i/lnIGVynCo-0nrA)

Pandas предоставляет нам много различных инструментов работы с табличными данными. Главные из них — это: класс таблицы `pandas.core.frame.DataFrame` и его методы; класс серии данных (например, столбец таблицы) `pandas.core.series.Series` и его методы; и различные функции библиотеки. 

Начнём с подгрузки таблицы из файла и посмотрим на методы класса `DataFrame`.

In [ ]:
df = pd.read_csv("grades.csv", sep=",", index_col=0)
type(df)

In [ ]:
# размер таблицы
df.shape

In [ ]:
# вывести начало таблицы
df.head(5)

In [ ]:
# вывести конец таблицы
df.tail()

In [ ]:
# выбрать случайные строки
df.sample(n=5)

In [ ]:
# выбрать случайные строки 2
df.sample(frac=0.3)

Добавим работу со столбцами: датафрейм можно индексировать квадратными скобками с названиями столбцов. Столбец будет экземпляром класса `pandas.core.series.Series`.

In [ ]:
col0 = df["0"]
col0

In [ ]:
type(col0)

In [ ]:
# выбрать 5 строк с наибольшими значениями в столбцах '3', '4'
df.nlargest(n=5, columns=["3", "4"])

In [ ]:
# подсчитать количество уникальных значений в столбце 3
df["3"].nunique()

In [ ]:
# подсчитать, сколько раз встретилось в столбце '3' каждое уникальное значение
df["3"].value_counts()

Доступ к столбцу можно получить не только через квадратные скобки, но и через точку, по аналогии с атрибутом класса.

In [ ]:
df["hash"]

In [ ]:
df.hash

А вот доступ к строкам можно получить при помощи `.iloc` и `loc`. Внешне они выглядят одинаково, но отличаются в деталях: первый индексирует по номеру строки, а второй — по индексирующему столбцу. Мы вернёмся к этому ниже, когда будет говорить про данные пассажиров Титаника. Обратите внимание, что такая индексация возвращает новый объект класса `DataFrame`.

In [ ]:
df.iloc[:5]

In [ ]:
# склеить две таблицы по строкам
df1 = df.iloc[:5]
df2 = df.iloc[10:15]
pd.concat([df1, df2])

In [ ]:
# склеить две таблицы по столбцам
df1 = df[["hash", "1"]]
df2 = df[["3", "4"]]
pd.concat([df1, df2], axis=1).head()

Метод `DataFrame.merge` объединяет таблицы по переданному столбцу (аналог SQL: JOIN).

In [ ]:
# таблица с оценками
df_grades = pd.read_csv("grades.csv", index_col=0)
df_grades.head()

In [ ]:
df_hashes = pd.read_csv("hashes.csv")
df_hashes.head()

Не для всех студентов известны оценки!

In [ ]:
df_grades.hash.nunique(), df_hashes.hash.nunique()

In [ ]:
# присоединить подходящие строки из df_grades к df_hashes
df = pd.merge(df_hashes, df_grades, on="hash", how="left")
print(df.shape)
df.head(10)

In [ ]:
# присоединить подходящие строки из df_hashes к df_grades
df = pd.merge(df_hashes, df_grades, on="hash", how="right")
print(df.shape)
df.head(10)

In [ ]:
# пересечение таблиц
# в данном случае эквивалентно 'right', т.к. в df_grades нет таких хэшей, которые отсутствуют в df_hashes
df = pd.merge(df_hashes, df_grades, on="hash", how="inner")
print(df.shape)
df.head(10)

In [ ]:
# объединение таблиц
# в данном случае эквивалентно 'left', т.к. в df_grades нет таких хэшей, которые отсутствуют в df_hashes
df = pd.merge(df_hashes, df_grades, on="hash", how="outer")
print(df.shape)
df.head(10)

In [ ]:
# выбрать из таблицы хэшей только те строки, в которых хэш есть в таблице оценок
# т.е. отобрать тех студентов, которые писали контрольную и были оценены
df = df_hashes[df_hashes.hash.isin(df_grades.hash)]
print(df.shape)
df.head()

In [ ]:
# сколько человек из каждой группы были оценены?
df.Группа.value_counts()

In [ ]:
type(df)

In [ ]:
type(df["Группа"])

Метод `DataFrame.groupby` делает группировку строк по значениям в каком-то столбце, чтобы мы могли их усреднить/просуммировать/etc. Например, если бы у нас была таблица вида [студент - предмет - оценка], то мы могли бы сгруппировать строки по столбцу предмета и посчитать по каждому предмету среднюю оценку.

In [ ]:
# сгруппировать строки по столбцу '1'
gr = df_grades.groupby(by="1")
gr

In [ ]:
df_grades.head()

In [ ]:
df_grades["1"].nunique()

In [ ]:
# какая средняя оценка (и ее дисперсия) за другие задачи у студентов, получивших конкретную оценку по задаче '1'?

In [ ]:
gr.mean()

In [ ]:
# дисперсия
gr.var()

In [ ]:
# откуда взялись NaN?

In [ ]:
gr.get_group(0.2)

## 2. Работаем с датасетом пассажиров Титаника

[Cсылка на скачивание](https://disk.yandex.ru/i/GWBoeAFjSMeiLg)

Каждая строчка наборов данных содержит следующие поля:

- Pclass — класс пассажира (1 — высший, 2 — средний, 3 — низший);

- Name — имя;

- Sex — пол;

- Age — возраст;

- SibSp — количество братьев, сестер, сводных братьев, сводных сестер, супругов на борту Титаника;

- Parch — количество родителей, детей (в том числе приемных) на борту Титаника;

- Ticket — номер билета;

- Fare — плата за проезд;

- Cabin — каюта;

- Embarked — порт посадки (C — Шербур; Q — Квинстаун; S — Саутгемптон)

- Survived - пассажир выжил или нет.

В поле Age приводится количество полных лет. Для детей меньше 1 года — дробное. Если возраст не известен точно, то указано примерное значение в формате xx.5.

In [ ]:
df = pd.read_csv("titanic_train.csv", sep=",")

In [ ]:
# типы данных
df.dtypes

In [ ]:
# сравним, сколько места занимает столбец
df["SibSp"].astype("int64").memory_usage()

In [ ]:
df["SibSp"].astype("int8").memory_usage()

In [ ]:
df.head()

Для первичного анализа полезно посмотреть на базовые статистики численных переменных. Для этого есть готовый метод:

In [ ]:
df.describe()

Также полезно проверить, какие переменные коррелированы больше, а какие — меньше.

In [ ]:
df.corr()

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.index

In [ ]:
Y = df["Survived"]
Y.head()

In [ ]:
df[["Survived", "Age"]].head()

In [ ]:
# удаление данных одного из столбцов
X = df[df.columns.drop("Survived")]

In [ ]:
X = df.drop("Survived", axis=1)  # same thing as previous cell

In [ ]:
df.shape, X.shape, Y.shape

Обещанная индексация! Давайте сперва проиндексируем таблицу двумя способами через `iloc` и `loc`, а затем изменим индексирующий столбец и посмотри на разницу.

In [ ]:
X.iloc[[5, 8, 10]]

In [ ]:
X.loc[[5, 8, 10]]

In [ ]:
# Меняем индексацию!
df_new = df.set_index("Name")
df_new.head()

In [ ]:
df_new.loc["Braund, Mr. Owen Harris"]

### Анализ данных

In [ ]:
df.head()

In [ ]:
set(df["Sex"])

In [ ]:
len(set(df["Name"]))

In [ ]:
df["Sex"].value_counts()

In [ ]:
df.pivot_table("PassengerId", "Sex", "Survived", "count").plot(kind="bar", stacked=True);
# Какой вывод из полученных гистограмм?

In [ ]:
df.pivot_table("PassengerId", "Pclass", "Survived", "count").plot(
    kind="bar", stacked=True
);
# Какой вывод из полученных гистограмм?

In [ ]:
fig, axes = plt.subplots(ncols=2)
df.pivot_table("PassengerId", ["SibSp"], "Survived", "count").plot(
    ax=axes[0], title="SibSp"
)
df.pivot_table("PassengerId", ["Parch"], "Survived", "count").plot(
    ax=axes[1], title="Parch"
);

# Какой вывод из полученных графиков?

In [ ]:
df.plot(x="PassengerId", y="Fare", kind="bar")

### Обработка и преобразование данных

In [ ]:
df.head(7)

In [ ]:
# преобразуем текстовый признак "Пол" в числовые значения
df["DecodedSex"] = df["Sex"].map({"male": 1, "female": -1, "unknown": 0})
df.head(2)

In [ ]:
# добавим еще одну характеристику для каждого объекта датасета
def fun(age):
    return age / 100


df["NewAge"] = df["Age"].apply(fun)
df.head(2)

In [ ]:
# то же самое можно сделать с помощью лямбда функции
df["NewAge"] = df["Age"].apply(lambda age: age / 100)
df.head(2)

In [ ]:
df["NewAge"] = df["Age"] / 100
df.head(2)

**Важно!** Pandas, как и NumPy, является лишь обёрткой на питоне для вычислительно эффективных операций над большими данными. Любое использование циклов в pandas приводит к неэффективности кода. Методы `.apply` и `.map` **медленные**, потому что внутри в цикле применяют питоновскую функцию к элементам таблицы. Старайтесь всегда использовать более эффективные реализации (например, арифметические операторы над столбцами) и прибегать к `.apply` и `.map` только в крайнем случае!

In [ ]:
%%timeit
df["NewAge"] = df["Age"].apply(lambda age: age / 100)

In [ ]:
%%timeit
df["NewAge"] = df["Age"] / 100

In [ ]:
# выделим фамилию из данных
df["Surname"] = df["Name"].apply(lambda name: name.split(",")[0])  # option1
df.head()

In [ ]:
df.shape

In [ ]:
df.Surname.nunique()

In [ ]:
df["Surname"] = df["Name"].apply(lambda name: name[: name.find(",")])  # option 2

In [ ]:
df["Surname"].value_counts().head()

In [ ]:
df.values  # df -> numpy.array

In [ ]:
# Исследуем возраст пассажиров Титаника
df.groupby("Sex")["Age"].mean()

In [ ]:
# опять же, обращаем внимание на эффективное использование pandas
%timeit df.groupby("Sex")["Age"].mean()

In [ ]:
%timeit df.groupby("Sex")["Age"].apply(np.mean)

In [ ]:
df.groupby("Sex")["Age"].apply(lambda ages: np.mean(ages) ** 2)

In [ ]:
df.groupby("Survived")["Age"].apply(np.mean)

In [ ]:
# группировка по нескольким столбцам и агрегация нескольких полей сразу
df.groupby(["Sex", "Pclass"]).agg(avg=("Age", "mean"), avg_surv=("Survived", "mean"))

In [ ]:
# .mean -> .count
# В скольких семьях было больше трёх человек?
np.sum(df.groupby("Surname")["Name"].count() > 3)

In [ ]:
# Сколько семей, в которых минимальный возраст меньше 10 лет?
np.sum(df.groupby("Surname")["Age"].apply(min) < 10)

Снова про индексацию! Как и в numpy, можно индексировать значения булевыми масками.

In [ ]:
((df["Age"] > 10) & (df["Age"] < 20)).value_counts()

In [ ]:
# пассажиры, удовлетворяющие условию
df.loc[(df["Age"] > 10) & (df["Age"] < 20)]